### Load Data File

In [1]:
%matplotlib inline
import math
import matplotlib.pyplot as plt
from decimal import *
import numpy as np # linear algebra
import pandas as pd # read_csv and such
from io import StringIO # convert strings to buffers or something like that.
import os # for listing files in directory
import seaborn as sns
import scipy.stats as stats

pd.options.display.max_columns = 99
pd.options.display.max_rows = 99

In [67]:
flights_weather_path = '../data/flight_delays_2015/flights_weather.csv'
dtypes = {
    'ORIGIN_AIRPORT': 'str', 
    'DESTINATION_AIRPORT': 'str', 
    'IATA_CODE_x': 'str', 
    'origin_weather_station': 'str', 
    'IATA_CODE_y': 'str', 
    'destination_weather_station': 'str', 
    'OR_MAX': 'str', 
    'OR_MIN': 'str', 
    'OR_PRCP': 'str', 
    'DES_MAX': 'str', 
    'DES_MIN': 'str', 
    'DES_PRCP': 'str', 
    'OR_FRSHTT': 'str', 
    'DES_FRSHTT': 'str'
}
fw_df = pd.read_csv(flights_weather_path, dtype=dtypes, parse_dates=['DATE'])
fw_df = fw_df[['DATE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'WEATHER_DELAY', 'OR_TEMP', 'OR_VISIB',
               'OR_WDSP', 'OR_MXSPD', 'OR_SNDP', 'OR_PRCP', 'OR_GUST', 'OR_MAX', 'OR_MIN', 'OR_FOG',
               'OR_RAIN_DRIZZLE', 'OR_SNOW_ICE_PELLETS', 'OR_HAIL', 'OR_THUNDER', 'OR_TORNADO_FUNNEL_CLOUD',
               'DES_TEMP', 'DES_VISIB', 'DES_WDSP', 'DES_MXSPD', 'DES_SNDP', 'DES_PRCP', 'DES_GUST', 'DES_MAX',
               'DES_MIN', 'DES_FOG', 'DES_RAIN_DRIZZLE', 'DES_SNOW_ICE_PELLETS', 'DES_HAIL', 'DES_THUNDER',
               'DES_TORNADO_FUNNEL_CLOUD']]
fw_df.head()

,DATE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,WEATHER_DELAY,OR_TEMP,OR_VISIB,OR_WDSP,OR_MXSPD,OR_SNDP,OR_PRCP,OR_GUST,OR_MAX,OR_MIN,OR_FOG,OR_RAIN_DRIZZLE,OR_SNOW_ICE_PELLETS,OR_HAIL,OR_THUNDER,OR_TORNADO_FUNNEL_CLOUD,DES_TEMP,DES_VISIB,DES_WDSP,DES_MXSPD,DES_SNDP,DES_PRCP,DES_GUST,DES_MAX,DES_MIN,DES_FOG,DES_RAIN_DRIZZLE,DES_SNOW_ICE_PELLETS,DES_HAIL,DES_THUNDER,DES_TORNADO_FUNNEL_CLOUD
0,2015-01-01,ANC,SEA,NaN,35.1,7.5,3.6,6.0,999.9,0.09G,999.9,43.0,32.0,True,True,False,False,False,False,32.9,10.0,4.6,11.1,999.9,0.00G,999.9,42.1,26.1,False,False,False,False,False,False
1,2015-01-01,ANC,SEA,NaN,35.1,7.5,3.6,6.0,999.9,0.09G,999.9,43.0,32.0,True,True,False,False,False,False,32.9,10.0,4.6,11.1,999.9,0.00G,999.9,42.1,26.1,False,False,False,False,False,False
2,2015-01-01,ANC,SEA,NaN,35.1,7.5,3.6,6.0,999.9,0.09G,999.9,43.0,32.0,True,True,False,False,False,False,32.9,10.0,4.6,11.1,999.9,0.00G,999.9,42.1,26.1,False,False,False,False,False,False
3,2015-01-01,ANC,SEA,NaN,35.1,7.5,3.6,6.0,999.9,0.09G,999.9,43.0,32.0,True,True,False,False,False,False,32.9,10.0,4.6,11.1,999.9,0.00G,999.9,42.1,26.1,False,False,False,False,False,False
4,2015-01-01,ANC,SEA,NaN,35.1,7.5,3.6,6.0,999.9,0.09G,999.9,43.0,32.0,True,True,False,False,False,False,32.9,10.0,4.6,11.1,999.9,0.00G,999.9,42.1,26.1,False,False,False,False,False,False


In [68]:
fw_df['OR_PRCP'] = pd.to_numeric(fw_df['OR_PRCP'].str.replace('A|B|C|D|E|F|G|H|I', ''))
fw_df['OR_MAX'] = pd.to_numeric(fw_df['OR_MAX'].str.replace('*', ''))
fw_df['OR_MIN'] = pd.to_numeric(fw_df['OR_MIN'].str.replace('*', ''))
fw_df.loc[fw_df['OR_TEMP'] == 9999.9, 'OR_TEMP'] = fw_df.OR_TEMP.mean()
fw_df.loc[fw_df['OR_WDSP'] == 999.9, 'OR_WDSP'] = fw_df.OR_WDSP.mean()
fw_df.loc[fw_df['OR_PRCP'] == 99.99, 'OR_PRCP'] = 0
fw_df.loc[fw_df['OR_VISIB'] == 999.9, 'OR_VISIB'] = fw_df.OR_VISIB.mean()
fw_df.loc[fw_df['OR_GUST'] == 999.9, 'OR_GUST'] = fw_df.OR_GUST.mean()
fw_df.loc[fw_df['OR_MAX'] == 9999.9, 'OR_MAX'] = fw_df.OR_MAX.mean()
fw_df.loc[fw_df['OR_MIN'] == 9999.9, 'OR_MIN'] = fw_df.OR_MIN.mean()
fw_df.loc[fw_df['OR_WDSP'] == 999.9, 'OR_WDSP'] = fw_df.OR_WDSP.mean()
fw_df.loc[fw_df['OR_MXSPD'] == 999.9, 'OR_MXSPD'] = fw_df.OR_MXSPD.mean()
fw_df.loc[fw_df['OR_SNDP'] == 999.9, 'OR_SNDP'] = 0
fw_df.loc[fw_df['OR_FOG'] == True, 'OR_FOGV'] = 1
fw_df.loc[fw_df['OR_FOG'] == False, 'OR_FOGV'] = 0
fw_df.loc[fw_df['OR_RAIN_DRIZZLE'] == True, 'OR_RAIN_DRIZZLEV'] = 1
fw_df.loc[fw_df['OR_RAIN_DRIZZLE'] == False, 'OR_RAIN_DRIZZLEV'] = 0
fw_df.loc[fw_df['OR_SNOW_ICE_PELLETS'] == True, 'OR_SNOW_ICE_PELLETSV'] = 1
fw_df.loc[fw_df['OR_SNOW_ICE_PELLETS'] == False, 'OR_SNOW_ICE_PELLETSV'] = 0
fw_df.loc[fw_df['OR_HAIL'] == True, 'OR_HAILV'] = 1
fw_df.loc[fw_df['OR_HAIL'] == False, 'OR_HAILV'] = 0
fw_df.loc[fw_df['OR_THUNDER'] == True, 'OR_THUNDERV'] = 1
fw_df.loc[fw_df['OR_THUNDER'] == False, 'OR_THUNDERV'] = 0
fw_df.loc[fw_df['OR_TORNADO_FUNNEL_CLOUD'] == True, 'OR_TORNADO_FUNNEL_CLOUDV'] = 1
fw_df.loc[fw_df['OR_TORNADO_FUNNEL_CLOUD'] == False, 'OR_TORNADO_FUNNEL_CLOUDV'] = 0
fw_df['DES_PRCP'] = pd.to_numeric(fw_df['DES_PRCP'].str.replace('A|B|C|D|E|F|G|H|I', ''))
fw_df['DES_MAX'] = pd.to_numeric(fw_df['DES_MAX'].str.replace('*', ''))
fw_df['DES_MIN'] = pd.to_numeric(fw_df['DES_MIN'].str.replace('*', ''))
fw_df.loc[fw_df['DES_TEMP'] == 9999.9, 'DES_TEMP'] = fw_df.DES_TEMP.mean()
fw_df.loc[fw_df['DES_WDSP'] == 999.9, 'DES_WDSP'] = fw_df.DES_WDSP.mean()
fw_df.loc[fw_df['DES_PRCP'] == 99.99, 'DES_PRCP'] = 0
fw_df.loc[fw_df['DES_VISIB'] == 999.9, 'DES_VISIB'] = fw_df.DES_VISIB.mean()
fw_df.loc[fw_df['DES_GUST'] == 999.9, 'DES_GUST'] = fw_df.DES_GUST.mean()
fw_df.loc[fw_df['DES_MAX'] == 9999.9, 'DES_MAX'] = fw_df.DES_MAX.mean()
fw_df.loc[fw_df['DES_MIN'] == 9999.9, 'DES_MIN'] = fw_df.DES_MIN.mean()
fw_df.loc[fw_df['DES_WDSP'] == 999.9, 'DES_WDSP'] = fw_df.DES_WDSP.mean()
fw_df.loc[fw_df['DES_MXSPD'] == 999.9, 'DES_MXSPD'] = fw_df.DES_MXSPD.mean()
fw_df.loc[fw_df['DES_SNDP'] == 999.9, 'DES_SNDP'] = 0
fw_df.loc[fw_df['DES_FOG'] == True, 'DES_FOGV'] = 1
fw_df.loc[fw_df['DES_FOG'] == False, 'DES_FOGV'] = 0
fw_df.loc[fw_df['DES_RAIN_DRIZZLE'] == True, 'DES_RAIN_DRIZZLEV'] = 1
fw_df.loc[fw_df['DES_RAIN_DRIZZLE'] == False, 'DES_RAIN_DRIZZLEV'] = 0
fw_df.loc[fw_df['DES_SNOW_ICE_PELLETS'] == True, 'DES_SNOW_ICE_PELLETSV'] = 1
fw_df.loc[fw_df['DES_SNOW_ICE_PELLETS'] == False, 'DES_SNOW_ICE_PELLETSV'] = 0
fw_df.loc[fw_df['DES_HAIL'] == True, 'DES_HAILV'] = 1
fw_df.loc[fw_df['DES_HAIL'] == False, 'DES_HAILV'] = 0
fw_df.loc[fw_df['DES_THUNDER'] == True, 'DES_THUNDERV'] = 1
fw_df.loc[fw_df['DES_THUNDER'] == False, 'DES_THUNDERV'] = 0
fw_df.loc[fw_df['DES_TORNADO_FUNNEL_CLOUD'] == True, 'DES_TORNADO_FUNNEL_CLOUDV'] = 1
fw_df.loc[fw_df['DES_TORNADO_FUNNEL_CLOUD'] == False, 'DES_TORNADO_FUNNEL_CLOUDV'] = 0
fw_df.loc[fw_df['WEATHER_DELAY'].isnull(), 'WEATHER_DELAY'] = 0


In [69]:
fw_df = fw_df[['DATE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'WEATHER_DELAY', 'OR_TEMP', 'OR_VISIB', 
               'OR_WDSP', 'OR_MXSPD', 'OR_SNDP', 'OR_PRCP', 'OR_MAX', 'OR_MIN', 'OR_FOGV',
               'OR_RAIN_DRIZZLEV', 'OR_SNOW_ICE_PELLETSV', 'OR_HAILV', 'OR_THUNDERV', 'OR_TORNADO_FUNNEL_CLOUDV',
               'DES_TEMP', 'DES_VISIB', 'DES_WDSP', 'DES_MXSPD', 'DES_SNDP', 'DES_PRCP', 'DES_MAX',
               'DES_MIN', 'DES_FOGV', 'DES_RAIN_DRIZZLEV', 'DES_SNOW_ICE_PELLETSV', 'DES_HAILV', 'DES_THUNDERV',
               'DES_TORNADO_FUNNEL_CLOUDV']]

In [70]:
fw_airports = fw_df.groupby(['DATE', 'ORIGIN_AIRPORT'], as_index=False).mean()

### Hypothesis Tests
Since our weather data is daily, it may not work to use it to predict delays of a specific flight, but it might be usable to predict average delays for an airport on a specific day. Since only the origin weather applies to all flights leaving an airport and and origin weather appears to have a stronger link to delays, I will focus on the origin airport.

Our null hypothesis would be that the mean weather conditions for flights with delays is the same as all flights. We can do this test for all of our weather variable, treating flights with a weather delay greater than 2 minutes as delayed:

#### Origin Precipitation

In [52]:
stats.ttest_1samp(a=fw_airports.loc[fw_airports['WEATHER_DELAY'] >= 1, 'OR_PRCP'], 
                 popmean=fw_airports[['OR_PRCP']].mean(), nan_policy='omit')

Ttest_1sampResult(statistic=OR_PRCP    12.750045
dtype: float64, pvalue=array([  6.24913584e-37]))

#### Origin Temp

In [53]:
stats.ttest_1samp(a=fw_airports.loc[fw_airports['WEATHER_DELAY'] >= 1, 'OR_TEMP'],
                 popmean=fw_airports[['OR_TEMP']].mean())

Ttest_1sampResult(statistic=OR_TEMP   -0.455533
dtype: float64, pvalue=array([ 0.6487364]))

#### Origin Max Temp

In [54]:
stats.ttest_1samp(a=fw_airports.loc[fw_airports['WEATHER_DELAY'] >= 1, 'OR_MAX'],
                 popmean=fw_airports[['OR_MAX']].mean())

Ttest_1sampResult(statistic=OR_MAX   -2.770157
dtype: float64, pvalue=array([ 0.00561381]))

#### Origin Min Temp

In [55]:
stats.ttest_1samp(a=fw_airports.loc[fw_airports['WEATHER_DELAY'] >= 1, 'OR_MIN'],
                 popmean=fw_airports[['OR_MIN']].mean())

Ttest_1sampResult(statistic=OR_MIN    2.581028
dtype: float64, pvalue=array([ 0.00986551]))

#### Origin Windspeed

In [56]:
stats.ttest_1samp(a=fw_airports.loc[fw_airports['WEATHER_DELAY'] >= 1, 'OR_WDSP'],
                 popmean=fw_airports[['OR_WDSP']].mean())

Ttest_1sampResult(statistic=OR_WDSP    11.998827
dtype: float64, pvalue=array([  6.23021173e-33]))

#### Origin Wind Gust

In [57]:
stats.ttest_1samp(a=fw_airports.loc[fw_airports['WEATHER_DELAY'] >= 1, 'OR_GUST'],
                 popmean=fw_airports[['OR_GUST']].mean())

Ttest_1sampResult(statistic=OR_GUST   -14.380283
dtype: float64, pvalue=array([  2.10881329e-46]))

#### Origin Visibility

In [58]:
stats.ttest_1samp(a=fw_airports.loc[fw_airports['WEATHER_DELAY'] >= 1, 'OR_VISIB'],
                 popmean=fw_airports[['OR_VISIB']].mean())

Ttest_1sampResult(statistic=OR_VISIB   -21.374135
dtype: float64, pvalue=array([  4.87633338e-99]))

#### Origin Fog

In [59]:
stats.ttest_1samp(a=fw_airports.loc[fw_airports['WEATHER_DELAY'] >= 1, 'OR_FOGV'],
                 popmean=fw_airports[['OR_FOGV']].mean())

Ttest_1sampResult(statistic=OR_FOGV    16.056905
dtype: float64, pvalue=array([  2.88909246e-57]))

#### Origin Rain or Drizzle

In [36]:
stats.ttest_1samp(a=fw_airports.loc[fw_airports['WEATHER_DELAY'] > 1, 'OR_RAIN_DRIZZLEV'],
                 popmean=fw_airports[['OR_RAIN_DRIZZLEV']].mean())

Ttest_1sampResult(statistic=OR_RAIN_DRIZZLEV    24.271907
dtype: float64, pvalue=array([  2.94130063e-126]))

#### Origin Snow/Ice/Pellets

In [37]:
stats.ttest_1samp(a=fw_airports.loc[fw_airports['WEATHER_DELAY'] > 1, 'OR_SNOW_ICE_PELLETSV'],
                 popmean=fw_airports[['OR_SNOW_ICE_PELLETSV']].mean())

Ttest_1sampResult(statistic=OR_SNOW_ICE_PELLETSV    17.388457
dtype: float64, pvalue=array([  1.12057466e-66]))

#### Origin Hail

In [38]:
stats.ttest_1samp(a=fw_airports.loc[fw_airports['WEATHER_DELAY'] >= 1, 'OR_HAILV'],
                 popmean=fw_airports[['OR_HAILV']].mean())

Ttest_1sampResult(statistic=OR_HAILV    5.588084
dtype: float64, pvalue=array([  2.35924658e-08]))

#### Origin Thunder

In [39]:
stats.ttest_1samp(a=fw_airports.loc[fw_airports['WEATHER_DELAY'] >= 1, 'OR_THUNDERV'],
                 popmean=fw_airports[['OR_THUNDERV']].mean())

Ttest_1sampResult(statistic=OR_THUNDERV    30.072963
dtype: float64, pvalue=array([  7.10681520e-190]))

#### Origin Tornado/Funnel Cloud

In [40]:
stats.ttest_1samp(a=fw_airports.loc[fw_airports['WEATHER_DELAY'] >= 1, 'OR_TORNADO_FUNNEL_CLOUDV'],
                 popmean=fw_airports[['OR_TORNADO_FUNNEL_CLOUDV']].mean())

Ttest_1sampResult(statistic=OR_TORNADO_FUNNEL_CLOUDV    1.926365
dtype: float64, pvalue=array([ 0.05408859]))

These pvalues still support the alternative hypothesis except in a single case, it looks as if average daily temp does not but that min and max temperature do.

In [41]:
from sklearn.cross_validation import train_test_split

/home/tcox/Envs/Volag/local/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [1]:
X_train, X_test, y_train, y_test = train_test_split(fw_airports[['OR_TEMP', 'OR_VISIB', 
               'OR_WDSP', 'OR_MXSPD', 'OR_SNDP', 'OR_PRCP', 'OR_GUST', 'OR_MAX', 'OR_MIN', 'OR_FOGV',
               'OR_RAIN_DRIZZLEV', 'OR_SNOW_ICE_PELLETSV', 'OR_HAILV', 'OR_THUNDERV', 'OR_TORNADO_FUNNEL_CLOUDV']], fw_airports[['WEATHER_DELAY']])


NameError: name 'train_test_split' is not defined

In [62]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train, y_train)

# print intercept and coefficients
print lm.intercept_
print lm.coef_

[ 6.02115643]
[[  1.59670244e-02  -6.66527716e-01   6.42369896e-02   3.73690261e-02
    3.73690261e-02  -2.38518914e-02  -2.38518914e-02   6.72083328e-02
    6.87893362e-04  -4.18784400e-04  -1.60542639e-02   2.07266464e+00
   -1.77432549e-01   1.90730300e+00   8.78974806e+00   2.26157793e+00
   -4.84879343e-01]]


In [64]:
lm.score(X_test, y_test)

0.018468063902115972